Notebook to test combinations of monitors. Testing aborted due to the performance of DE.

In [25]:
import pandas as pd
import ast

In [2]:
#Load ThirdEye and SelfOracle best configuations
icse= pd.read_csv('results\old\smoothgrad-icse20_t95.csv')
ood= pd.read_csv('results\old\smoothgrad-ood_t95.csv')
mutants= pd.read_csv('results\old\smoothgrad-mutants_t95.csv')

group1_df = pd.concat([icse, ood], ignore_index=True)
ty_95 = pd.concat([group1_df, mutants], ignore_index=True)

In [3]:
#Load ThirdEye and SelfOracle best configuations
icse= pd.read_csv('results\old\smoothgrad-icse20_t99.csv')
ood= pd.read_csv('results\old\smoothgrad-ood_t99.csv')
mutants= pd.read_csv('results\old\smoothgrad-mutants_t99.csv')

group1_df = pd.concat([icse, ood], ignore_index=True)
ty_99 = pd.concat([group1_df, mutants], ignore_index=True)

In [4]:
#Load ThirdEye and SelfOracle best configuations
icse= pd.read_csv(r'results\old\track1-MSE-latent2-selforacle-icse20-t95.csv')
ood= pd.read_csv(r'results\old\track1-MSE-latent2-selforacle-ood-t95.csv')
mutants= pd.read_csv(r'results\old\track1-MSE-latent2-selforacle-mutants-t95.csv')

group1_df = pd.concat([icse, ood], ignore_index=True)
df = pd.concat([group1_df, mutants], ignore_index=True)
df = df[df['aggregation_type']=='max']
so_95 = df.drop_duplicates()

In [5]:
#Load ThirdEye and SelfOracle best configuations
icse= pd.read_csv(r'results\old\track1-MSE-latent2-selforacle-icse20-t999.csv')
ood= pd.read_csv(r'results\old\track1-MSE-latent2-selforacle-ood-t999.csv')
mutants= pd.read_csv(r'results\old\track1-MSE-latent2-selforacle-mutants-t999.csv')

group1_df = pd.concat([icse, ood], ignore_index=True)
df = pd.concat([group1_df, mutants], ignore_index=True)
df = df[df['aggregation_type']=='max']
so_999 = df.drop_duplicates()

In [6]:
de3_999 = pd.read_csv(r'results\final_999\dave2-track1-DE_3.csv')
de5_9999 = pd.read_csv(r'results\final_9999\dave2-track1-DE_5.csv')
mcd5_S64_999 = pd.read_csv(r'results\final_999\dave2-p10-track1-mcd_5_S64.csv')
mcd5_S123_99999 = pd.read_csv(r'results\final_99999\dave2-p10-track1-mcd_5_S128.csv')

In [14]:
# List of strings to replace
# Strings to replace
strings_to_replace = ['icse20\\', 'ood\\', 'mutants\\']

# Escape the backslash (\) in the strings for regex
strings_to_replace = [s.replace('\\', '\\\\') for s in strings_to_replace]

# Replace the strings with an empty string
ty_95['simulation_name'] = ty_95['simulation_name'].replace(strings_to_replace, '', regex=True)
ty_99['simulation_name'] = ty_99['simulation_name'].replace(strings_to_replace, '', regex=True)
so_95['simulation_name'] = so_95['simulation_name'].replace(strings_to_replace, '', regex=True)
so_999['simulation_name'] = so_999['simulation_name'].replace(strings_to_replace, '', regex=True)

In [15]:
group1_dfs = [so_95, so_999]
group2_dfs = [mcd5_S64_999, mcd5_S123_99999]

In [16]:
so_95.name = 'so_95'
so_999.name = 'so_999'
mcd5_S64_999.name = 'mcd5_S64_999'
mcd5_S123_99999.name = 'mcd5_S123_99999'

In [27]:
# Initialize a list to store the merged DataFrames
merged_dfs = []

# Iterate through the combinations of DataFrames
for i, df1 in enumerate(group1_dfs):
    for j, df2 in enumerate(group2_dfs):
        # Merge DataFrames based on 'simulation_name' and 'ttm'
        merged_df = df1.merge(df2, on=['simulation_name', 'ttm'], suffixes=('_model1', '_model2'))
        
        # Convert the columns to sets of integers
        merged_df['detected_failure_ids_model1'] = merged_df['detected_failure_ids_model1'].apply(lambda x: set(map(int, ast.literal_eval(x))))
        merged_df['detected_failure_ids_model2'] = merged_df['detected_failure_ids_model2'].apply(lambda x: set(map(int, ast.literal_eval(x))))
        merged_df['undetected_failure_ids_model1'] = merged_df['undetected_failure_ids_model1'].apply(lambda x: set(map(int, ast.literal_eval(x))))
        merged_df['undetected_failure_ids_model2'] = merged_df['undetected_failure_ids_model2'].apply(lambda x: set(map(int, ast.literal_eval(x))))

        # Create the 'undetected_failure_ids' column
        def remove_detected_undetected(row):
            # Get unique values in 'FailureIDs_model1'
            failure_ids_model1 = set(row['FailureIDs_model1'])
            
            # Remove detected and undetectable failures from 'FailureIDs_model1'
            for col_suffix in ['model1', 'model2']:
                detected_col = f'detected_failure_ids_{col_suffix}'
                undetectable_col = f'undetectable_failure_ids_{col_suffix}'
                # Remove duplicates while removing detected and undetectable failures
                failure_ids_model1 = list(set(failure_ids_model1) - set(row[detected_col]) - set(row[undetectable_col]))
            
            # Filter out non-integer values from the 'undetected_failure_ids' column
            undetected_failure_ids = [value for value in failure_ids_model1 if str(value).isdigit()]
            
            return undetected_failure_ids

        # Apply the function to create the 'undetected_failure_ids' column
        merged_df['undetected_failure_ids'] = merged_df.apply(remove_detected_undetected, axis=1)

        selected_columns = ['model', 'simulation_name', 'ttm', 'FailureIDs_model1','FailureIDs_model2', 'detected_failure_ids_model1', 'detected_failure_ids_model2', 'undetected_failure_ids_model1', 'undetected_failure_ids_model2']

        merged_df = merged_df[selected_columns]

        def common_undetected_ids(row):
            # Find the common integers
            common_ids = list(row['undetected_failure_ids_model1'].intersection(row['undetected_failure_ids_model2']))
            return common_ids

        merged_df['common_undetected_ids'] = merged_df.apply(common_undetected_ids, axis=1)
        merged_df = merged_df[merged_df['ttm'] == 3]

        # Append the merged DataFrame to the list
        merged_dfs.append(merged_df)

        # Save the merged DataFrame to a CSV file
        file_name = f"hybrid/{df1.name}_{df2.name}_merged.csv"
        merged_df.to_csv(file_name, index=False)


In [28]:
import pandas as pd

# ... (your previous code to merge DataFrames)

# Create a new DataFrame with the relevant columns
contingency_df = merged_df[['undetected_failure_ids_model1', 'undetected_failure_ids_model2', 'detected_failure_ids_model1', 'detected_failure_ids_model2']]

# Convert frame lists to the count of frames
contingency_df['undetected_failure_ids_model1'] = contingency_df['undetected_failure_ids_model1'].apply(len)
contingency_df['undetected_failure_ids_model2'] = contingency_df['undetected_failure_ids_model2'].apply(len)
contingency_df['detected_failure_ids_model1'] = contingency_df['detected_failure_ids_model1'].apply(len)
contingency_df['detected_failure_ids_model2'] = contingency_df['detected_failure_ids_model2'].apply(len)

# Calculate the total sums of frame counts
undetected_model1_sum = contingency_df['undetected_failure_ids_model1'].sum()
undetected_model2_sum = contingency_df['undetected_failure_ids_model2'].sum()
detected_model1_sum = contingency_df['detected_failure_ids_model1'].sum()
detected_model2_sum = contingency_df['detected_failure_ids_model2'].sum()

# Create a single contingency matrix
contingency_matrix = pd.DataFrame([[undetected_model1_sum, undetected_model2_sum], [detected_model1_sum, detected_model2_sum]],
                                  columns=['Model1', 'Model2'],
                                  index=['Undetected', 'Detected'])

# The contingency_matrix variable now contains the single contingency matrix
print(contingency_matrix)

            Model1  Model2
Undetected     190      31
Detected        75     261


c:\Users\ruben\anaconda3\envs\udacity-self-driving-car\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
c:\Users\ruben\anaconda3\envs\udacity-self-driving-car\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
c:\Users\ruben\anaconda3\envs\udacity-self-driving-car\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to